# Pytorch Tutorial

In [1]:
import torch
import torch.nn as nn

## Tenzorok

Az adatokat tenzorok reprezentálják. A tenzorok kezelése hasonló a numpy-hoz bár, a függvények nevei eltérnek.

### tenzor létrehozása

In [3]:
a = torch.tensor((5, 6), dtype=torch.float32)  # itt is sor, oszlop
print(a)

In [5]:
b = torch.eye(4)
print(b)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [6]:
c = torch.zeros((3, 2))
print(c)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [8]:
d = torch.ones((3, 7), dtype=bool)
print(d)

tensor([[True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True]])


In [9]:
e = torch.normal(mean=torch.zeros(3, 3), std=torch.ones(3, 3))
print(e)

tensor([[-0.9434, -2.2135,  0.2787],
        [ 1.4550, -1.1730, -1.1258],
        [ 1.0159,  0.7988, -2.1531]])


In [10]:
# ha rendelkezésre áll másik hardware is, akkor a tensort át lehet tenni gpu-ra illetve vissza
f = torch.as_tensor([5, 6, 8], dtype=torch.int32, device=torch.device('cuda'))
print(f)

tensor([5, 6, 8], device='cuda:0', dtype=torch.int32)


In [11]:
# visszafelé
g = f.cpu().detach().numpy()
print(g)

[5 6 8]


### műveletek

In [15]:
# összeadás
a = torch.normal(mean=torch.zeros(3, 3), std=torch.ones(3, 3))
b = torch.normal(mean=torch.zeros(3, 3), std=torch.ones(3, 3))
print(a + b)

tensor([[ 0.2077, -0.0185,  0.2137],
        [ 0.1829, -1.5721,  1.7716],
        [-0.7753,  1.1038,  1.3150]])


In [16]:
# redukció
a.sum(dim=0)  # axis helyett dim van

tensor([0.5054, 2.9784, 4.0083])

In [17]:
a.max()

tensor(1.9365)

In [20]:
aa = a.unsqueeze(0)  # hozzáad egy plusz dimenziót, dim=0-nál
aa.shape

torch.Size([1, 3, 3])

In [21]:
aa = a.unsqueeze(1)
aa.shape

torch.Size([3, 1, 3])

In [23]:
# vektorizáció
print(a)
torch.log(a)  # nan ott lehet, ahol negatív a bemenet

tensor([[-0.3171,  1.3075,  1.6865],
        [ 0.3816, -0.2656,  1.5144],
        [ 0.4409,  1.9365,  0.8073]])


tensor([[    nan,  0.2681,  0.5227],
        [-0.9634,     nan,  0.4151],
        [-0.8190,  0.6609, -0.2140]])

## Példa az MNIST-en

In [2]:
from pckutils import mnist
mnist_data = mnist.load_mnist('data')

2051 60000 28 28
Reading images: [100%]
2049 60000
Reading labels: [100%]
2051 10000 28 28
Reading images: [100%]
2049 10000
Reading labels: [100%]


In [3]:
#reshape the images
reshape = lambda img: img.reshape((mnist_data.rows, mnist_data.cols))
mnist_data.X_train = [reshape(img) for img in mnist_data.X_train]
mnist_data.X_test = [reshape(img) for img in mnist_data.X_test]

### Adat kezelés

Az alábbiakra fogunk figyelni az adatbetöltés során:

* legyen tanításhoz és validációhoz is adat
* augmentációt is fogunk használni a batchek mintavételezése közben

In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

In [5]:
# defining paramters
batch_size = 32
split_ratio = 0.8
device = torch.device("cuda")  # "cpu"

In [7]:
# ez a Dataset, amit a Pytorch saját Loader-e vár
class MnistDataset(Dataset):
    def __init__(self, device, images, labels, transform=lambda x: np.expand_dims(x, axis=0)):  # a transformer olyan képet ad, aminek már van channelje
        self.device = device
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_ = self.images[index]
        image = self.transform(image_)  # itt alkalmazzuk az augmentciót
        label = self.labels[index]
        sample = {
            'image_id': index,
            'image': torch.tensor(image, dtype=torch.float, device=self.device),
            'target': torch.tensor(label, dtype=torch.long, device=self.device)
        }
        return sample

# definiáljuk az adatbetöltőket

# felbontjuk azadatot
indices = list(range(mnist_data.num_train_imgs))
np.random.shuffle(indices)

split_index = int(len(indices) * split_ratio)
idices_for_train = indices[0:split_index]
indices_for_validation = indices[split_index:]

# Training loader
training_img = [mnist_data.X_train[idx] for idx in idices_for_train]  # list of images
training_label = [mnist_data.Y_train[idx] for idx in idices_for_train]

augmenter = transforms.Compose(  # tipikus augmentáció tartalmazza: forgatás, nyújtás, vágás, traszláció, zaj, kontraszt, fényesség stb.
    [   transforms.ToPILImage(),
        transforms.RandomAffine([-90, 90]),  # forgatás
        transforms.ToTensor()
    ]
)
dataset = MnistDataset(device, training_img, training_label, augmenter)
loader_train = DataLoader(dataset, batch_size)

# Validation loader
validation_img = [mnist_data.X_train[idx] for idx in indices_for_validation]  # képek listája
validation_label = [mnist_data.Y_train[idx] for idx in indices_for_validation]

dataset = MnistDataset(device, validation_img, validation_label)
loader_validation = DataLoader(dataset, batch_size)

# Test loader
test_img = mnist_data.X_test  # képek listája
test_label = mnist_data.Y_test

dataset = MnistDataset(device, test_img, test_label)
loader_test = DataLoader(dataset, 1)  # egyenként fogjuk tesztelni

### Modell építés

In [8]:
class MNISTclassifier(nn.Module):
    def __init__(self):
        super(MNISTclassifier, self).__init__()
        # creating the network architecture

        # expected input size: (batch, 1, 28, 28)
        self.conv1_1 = nn.Conv2d(1, 8, (3, 3), stride=1)     # (., 8, 26, 26)
        self.conv1_2 = nn.Conv2d(8, 16, (3, 3), stride=1)    # (., 16, 24, 24)
        self.conv1_3 = nn.Conv2d(16, 16, (2, 2), stride=2)    # (., 16, 12, 12)
        self.conv1_4 = nn.Conv2d(16, 8, (3, 3), stride=1)   # (., 8, 10, 10)
        
        self.linear = nn.Linear(800, 10)

        self.relu = nn.ReLU()

    def forward(self, x):
        temp = self.relu(self.conv1_1(x))
        temp = self.relu(self.conv1_2(temp))
        temp = self.relu(self.conv1_3(temp))
        temp = self.relu(self.conv1_4(temp))
        temp = temp.view(-1, 800)
        temp = self.linear(temp)
        return temp

In [9]:
# példányosítsunk is egyet
model = MNISTclassifier()
model.to(device)

MNISTclassifier(
  (conv1_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv1_2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv1_3): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
  (conv1_4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1))
  (linear): Linear(in_features=800, out_features=10, bias=True)
  (relu): ReLU()
)

### A tanítás folyamata

In [10]:
import torch.optim as optim
from datetime import datetime

epochs = 6
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [11]:
print("Started training at {}".format(datetime.now()))

global_counter = 0

def _validation(epoch, index, global_counter):
    counter = 0
    loss_sum = 0.0
    for sample in loader_validation:
        counter += 1
        image = sample['image']
        target = sample['target']
        predicted = model(image)
        loss = criterion(predicted, target)
        loss_sum += loss.cpu().detach().numpy()  # a kiszámolt loss is a gpu-n van, vissza kell hozni
        del loss

    loss_mean = loss_sum / counter
    print("Current status at: (epoch: %d, i: %d) with validation loss: %f"%(epoch, index, loss_mean))

for epoch in range(epochs):
    for index, sample in enumerate(loader_train):
        global_counter += 1

        image = sample['image']
        target = sample['target']
        
        predicted = model(image)
        loss = criterion(predicted, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss = loss.cpu().detach().numpy()

    _validation(epoch, index, global_counter)
    print("\r Current epoch {} and global counter {}".format(epoch, global_counter), end="")
print("Training has finished.")

Started training at 2020-11-08 22:20:18.965081


c:\users\adambudai\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Current status at: (epoch: 0, i: 1499 ) with validation loss: 41.990800
 Current epoch 0 and global counter 1500Current status at: (epoch: 1, i: 1499 ) with validation loss: 40.692638
 Current epoch 1 and global counter 3000Current status at: (epoch: 2, i: 1499 ) with validation loss: 33.464447
 Current epoch 2 and global counter 4500Current status at: (epoch: 3, i: 1499 ) with validation loss: 34.656633
 Current epoch 3 and global counter 6000Current status at: (epoch: 4, i: 1499 ) with validation loss: 34.829066
 Current epoch 4 and global counter 7500Current status at: (epoch: 5, i: 1499 ) with validation loss: 26.592075
 Current epoch 5 and global counter 9000Training has finished.


### Eredmény tesztelése

In [12]:
# megszámoljuk hány esetben ad jó eredményt a megoldás
counter = 0
correctly_labeled = 0
for sample in loader_test:
    counter += 1
    image = sample['image']
    target = sample['target']
    predicted = torch.argmax(model(image))

    if predicted == target:
        if (counter%500 == 0):
            print(predicted.cpu().detach().numpy(), target[0].cpu().detach().numpy())
        correctly_labeled += 1

print("Correctly labeled {} out of {}".format(correctly_labeled, counter))

6 6
9 9
8 8
5 5
4 4
0 0
8 8
9 9
6 6
0 0
1 1
7 7
2 2
4 4
9 9
8 8
6 6
0 0
1 1
6 6
Correctly labeled 9429 out of 10000


### Modell mentése

In [13]:
path = 'minst_weights.pt'
checkpoint = {
    'mnist_state': model.state_dict(),
}
torch.save(checkpoint, path)

In [14]:
# visszatöltés
model2 = MNISTclassifier()
checkpoint = torch.load(path)
model2.load_state_dict(checkpoint['mnist_state'])
model2.to(device)

MNISTclassifier(
  (conv1_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv1_2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv1_3): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
  (conv1_4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1))
  (linear): Linear(in_features=800, out_features=10, bias=True)
  (relu): ReLU()
)